In [75]:
import os
import json
from functools import partial

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [12]:
DATA_PATH = '/home/zanoni/Documentos/fer_plus_project/face-emotion/data/'
LABELS_FNAME = 'label.csv'
TRAIN_PATH = os.path.join(DATA_PATH, 'FER2013Train')
VAL_PATH = os.path.join(DATA_PATH, 'FER2013Valid')
TEST_PATH = os.path.join(DATA_PATH, 'FER2013Test')

In [9]:
LABELS = [
    'neutral',
    'happiness',
    'surprise',
    'sadness',
    'anger',
    'disgust',
    'fear',
    'contempt',
    'unknown',
    'not_a_face'
]
DF_COLUMNS = ['img_name', 'bbox', *LABELS]
N_CLASSES = len(LABELS)

## Data Analysis
Perform a small Exploratory Data Analysis on the train label CSVs, in order to check the labels distribution and bboxes.

In [48]:
df = pd.read_csv(
    os.path.join(TRAIN_PATH, LABELS_FNAME),
    header=None, index_col=None, names=DF_COLUMNS)
df.head()

,img_name,bbox,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,not_a_face
0,fer0000000.png,"(0, 0, 48, 48)",4,0,0,1,3,2,0,0,0,0
1,fer0000001.png,"(0, 0, 48, 48)",6,0,1,1,0,0,0,0,2,0
2,fer0000002.png,"(0, 0, 48, 48)",5,0,0,3,1,0,0,0,1,0
3,fer0000003.png,"(0, 0, 48, 48)",4,0,0,4,1,0,0,0,1,0
4,fer0000004.png,"(0, 0, 48, 48)",9,0,0,1,0,0,0,0,0,0


In [49]:
# expand the image names to the full path
df['img_name'] = TRAIN_PATH + os.sep + df['img_name']

In [50]:
# generate the label based on the annotations
df['label'] = np.argmax(df.iloc[:, -N_CLASSES:], axis=1)
df['label'].value_counts()

label
0    10295
1     7526
2     3557
3     3530
4     2463
6      655
5      191
8      171
7      168
9        2
Name: count, dtype: int64

In [51]:
df['bbox'].value_counts()

bbox
(0, 0, 48, 48)    28558
Name: count, dtype: int64

In [52]:
# remove redundant columns
# since all bboxes have the same value, they all can be removed
df = df.drop(LABELS + ['bbox'], axis=1)

In [53]:
df.head()

,img_name,label
0,/home/zanoni/Documentos/fer_plus_project/face-...,0
1,/home/zanoni/Documentos/fer_plus_project/face-...,0
2,/home/zanoni/Documentos/fer_plus_project/face-...,0
3,/home/zanoni/Documentos/fer_plus_project/face-...,0
4,/home/zanoni/Documentos/fer_plus_project/face-...,0


## Create TF Dataset

In [126]:
ds = tf.data.Dataset.from_tensor_slices((df['img_name'], df['label']))
next(iter(ds))

(<tf.Tensor: shape=(), dtype=string, numpy=b'/home/zanoni/Documentos/fer_plus_project/face-emotion/data/FER2013Train/fer0000000.png'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

In [127]:
ds = ds.shuffle(df.shape[0])
next(iter(ds))

(<tf.Tensor: shape=(), dtype=string, numpy=b'/home/zanoni/Documentos/fer_plus_project/face-emotion/data/FER2013Train/fer0028582.png'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>)

In [128]:
@tf.numpy_function(Tout=(tf.uint8, tf.int64))
def read_gs_image(img_path, label):
    return cv2.imread(img_path.decode("utf-8"), 0), label

ds = ds.map(read_gs_image, num_parallel_calls=tf.data.AUTOTUNE)
next(iter(ds))

(<tf.Tensor: shape=(48, 48), dtype=uint8, numpy=
 array([[74, 78, 70, ..., 94, 94, 90],
        [80, 88, 64, ..., 94, 94, 91],
        [57, 64, 65, ..., 95, 94, 92],
        ...,
        [35, 35, 27, ..., 32, 32, 49],
        [32, 32, 26, ..., 31, 32, 31],
        [31, 28, 23, ..., 26, 26, 28]], dtype=uint8)>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>)

In [129]:
ds = ds.batch(32)
next(iter(ds))

(<tf.Tensor: shape=(32, 48, 48), dtype=uint8, numpy=
 array([[[109,  52,  36, ...,  98,  98,  99],
         [ 85,  36,  59, ...,  95,  99, 100],
         [ 61,  59,  66, ...,  84, 102, 101],
         ...,
         [ 98,   0,  40, ...,  14,  30,  43],
         [ 12,  29, 190, ...,  10,  38,  49],
         [  0,  28,  77, ...,   4,   0,  46]],
 
        [[242, 240, 248, ...,  33,  24,  17],
         [242, 240, 248, ...,  21,  20,  19],
         [242, 240, 249, ...,  20,  14,  13],
         ...,
         [235, 235, 235, ..., 233, 232, 232],
         [235, 235, 235, ..., 231, 232, 231],
         [235, 235, 235, ..., 239, 231, 231]],
 
        [[166, 166, 166, ..., 135, 136, 136],
         [167, 166, 167, ..., 138, 139, 138],
         [169, 166, 163, ..., 139, 140, 139],
         ...,
         [ 25,  21,  48, ..., 132, 131, 129],
         [ 27,  24,  18, ..., 134, 133, 131],
         [ 29,  27,  24, ..., 135, 134, 132]],
 
        ...,
 
        [[241, 194,  73, ..., 255, 255, 255],
       

In [130]:
@tf.function
def normalize_img(img_batch, label_batch):
    # normalize image to the range of [-1, 1]
    return 2*(img_batch / 255) - 1, label_batch

ds = ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
next(iter(ds))

(<tf.Tensor: shape=(32, 48, 48), dtype=float32, numpy=
 array([[[-0.5294118 , -0.7176471 , -0.5137255 , ..., -0.29411763,
          -0.3333333 , -0.372549  ],
         [-0.67058825, -0.67058825, -0.4823529 , ..., -0.26274508,
          -0.30196077, -0.32549018],
         [-0.78039217, -0.6392157 , -0.47450978, ..., -0.24705881,
          -0.26274508, -0.29411763],
         ...,
         [ 0.02745104,  0.18431377,  0.4901961 , ..., -0.01176471,
          -0.00392157, -0.08235294],
         [ 0.30196083,  0.14509809,  0.17647064, ...,  0.02745104,
          -0.06666666, -0.19999999],
         [ 0.34901965,  0.32549024,  0.17647064, ..., -0.0745098 ,
          -0.19215685, -0.16862744]],
 
        [[-0.54509807, -0.52156866, -0.5058824 , ..., -0.2862745 ,
          -0.2862745 , -0.29411763],
         [-0.5294118 , -0.52156866, -0.5137255 , ..., -0.23921567,
          -0.2235294 , -0.24705881],
         [-0.5137255 , -0.5058824 , -0.5137255 , ..., -0.25490195,
          -0.2235294 , -0.223

In [125]:
@tf.function
def label_to_one_hot(img_batch, label_batch, n_classes: int = 10):
    return img_batch, tf.one_hot(label_batch, n_classes)

ds = ds.map(
    partial(label_to_one_hot, n_classes=N_CLASSES),
    num_parallel_calls=tf.data.AUTOTUNE)
next(iter(ds))

(<tf.Tensor: shape=(32, 48, 48), dtype=uint8, numpy=
 array([[[101, 104, 113, ..., 251, 250, 250],
         [138, 137, 160, ..., 249, 250, 253],
         [191, 166, 140, ..., 246, 248, 253],
         ...,
         [112,  74,  58, ..., 216, 220, 218],
         [118, 103,  85, ..., 224, 225, 219],
         [117, 143, 126, ..., 227, 225, 218]],
 
        [[  0,   0,   0, ...,  15,   2,   1],
         [  0,   0,   0, ...,  17,   4,   0],
         [  0,   0,   0, ...,   1,  10,   6],
         ...,
         [  6,   6,   8, ...,   0,   0,   0],
         [  1,   0,   0, ...,   0,   0,   0],
         [  0,   0,   2, ...,   0,   0,   0]],
 
        [[ 68,  69,  72, ...,  60,  58,  58],
         [ 66,  65,  61, ...,  62,  61,  60],
         [ 67,  64,  59, ...,  63,  63,  61],
         ...,
         [ 31,  39,  47, ...,  62,  58,  57],
         [ 35,  41,  45, ...,  59,  63,  58],
         [ 38,  44,  45, ...,  54,  68,  62]],
 
        ...,
 
        [[159, 135, 106, ..., 187, 200, 209],
       